# Exercise 1.2.1: Huffman Coding

In [113]:
import numpy as np
import sys
sys.path.append("../../..")
from src.compression.entropy import entropy

In [114]:
class HuffmanNode:
    def __init__(self, character = None, frequency = 0):
        self.character = character
        self.frequency = frequency
        self.left = None
        self.right = None

    
    def __repr__(self):
        return f"Node=({self.character}, freq={self.frequency})"
    

def convert_string_to_freq(string):
    character_freq_dict = {}
    characterlist = list(string)
    for char in range(0, len(characterlist)):
        if characterlist[char] not in character_freq_dict:
            character_freq_dict[characterlist[char]] = 1
        elif characterlist[char] in character_freq_dict:
            character_freq_dict[characterlist[char]] += 1
    
    return character_freq_dict

def get_Char(item):
    return item.character if isinstance(item, HuffmanNode) else item

def create_huffman_tree(characterset):
    freqdict = convert_string_to_freq(characterset)
    nodes_required = len(freqdict)
    sorted_dict = dict(sorted(freqdict.items(), key = lambda x:x[1]))
    for i in range(0, nodes_required-1):
        sorted_dict = dict(sorted(sorted_dict.items(), key = lambda x:x[1]))
        x = next(iter(sorted_dict.items()))
        sorted_dict.pop(x[0])
        y = next(iter(sorted_dict.items()))
        sorted_dict.pop(y[0])
        node = HuffmanNode(character=f"{get_Char(x[0])}, {get_Char(y[0])}", frequency=(x[1] + y[1]))
        node.left = x[0]
        node.right = y[0]
        sorted_dict[node] = node.frequency
    
    tree = next(iter(sorted_dict.items()))[0]
    return tree

def show_huffman_tree(huffmantree, prefix="", is_left=True):
    if huffmantree is None:
        return
    
    # Check if this is a leaf node (left/right are strings) or internal node
    if isinstance(huffmantree, HuffmanNode):
        print(f"{prefix}{'L-- ' if is_left else 'R-- '}{huffmantree}")
        
        # Recursively show left subtree
        if huffmantree.left:
            if isinstance(huffmantree.left, HuffmanNode):
                show_huffman_tree(huffmantree.left, prefix + ("    " if is_left else "│   "), True)
            else:
                print(f"{prefix}    L-- Leaf: {huffmantree.left}")
        
        # Recursively show right subtree
        if huffmantree.right:
            if isinstance(huffmantree.right, HuffmanNode):
                show_huffman_tree(huffmantree.right, prefix + ("    " if is_left else "│   "), False)
            else:
                print(f"{prefix}    R-- Leaf: {huffmantree.right}")








In [115]:
set1 = "AAAAAABBBBBBBCCCDDE"
print(convert_string_to_freq(set1))
huffmantree1 = create_huffman_tree(set1)
print(huffmantree1)

{'A': 6, 'B': 7, 'C': 3, 'D': 2, 'E': 1}
Node=(B, A, C, E, D, freq=19)


In [116]:
show_huffman_tree(huffmantree1)

L-- Node=(B, A, C, E, D, freq=19)
    L-- Leaf: B
    R-- Node=(A, C, E, D, freq=12)
        L-- Leaf: A
    │   R-- Node=(C, E, D, freq=6)
    │       L-- Leaf: C
    │   │   R-- Node=(E, D, freq=3)
    │   │       L-- Leaf: E
    │   │       R-- Leaf: D


In [117]:
set2 = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum"
huffmantree2 = create_huffman_tree(set2)
print(convert_string_to_freq(set2))



{'L': 1, 'o': 29, 'r': 22, 'e': 37, 'm': 17, ' ': 68, 'i': 42, 'p': 11, 's': 18, 'u': 28, 'd': 18, 'l': 21, 't': 32, 'a': 29, ',': 4, 'c': 16, 'n': 24, 'g': 3, 'b': 3, 'q': 5, '.': 3, 'U': 1, 'v': 3, 'x': 3, 'D': 1, 'h': 1, 'f': 3, 'E': 1}


In [118]:
print(huffmantree2)
show_huffman_tree(huffmantree2)

Node=(,, q, p, l, i, r, D, h, g, b, ., v, x, f, E, L, U, n, u, o, a, t, c, m,  , s, d, e, freq=444)
L-- Node=(,, q, p, l, i, r, D, h, g, b, ., v, x, f, E, L, U, n, u, o, a, t, c, m,  , s, d, e, freq=444)
    L-- Node=(,, q, p, l, i, r, D, h, g, b, ., v, x, f, E, L, U, n, u, freq=180)
        L-- Node=(,, q, p, l, i, freq=83)
            L-- Node=(,, q, p, l, freq=41)
                L-- Node=(,, q, p, freq=20)
                    L-- Node=(,, q, freq=9)
                        L-- Leaf: ,
                        R-- Leaf: q
                    R-- Leaf: p
                R-- Leaf: l
            R-- Leaf: i
        R-- Node=(r, D, h, g, b, ., v, x, f, E, L, U, n, u, freq=97)
        │   L-- Node=(r, D, h, g, b, ., v, x, f, E, L, U, freq=45)
        │       L-- Leaf: r
        │       R-- Node=(D, h, g, b, ., v, x, f, E, L, U, freq=23)
        │       │   L-- Node=(D, h, g, b, ., freq=11)
        │       │       L-- Node=(D, h, g, freq=5)
        │       │           L-- Node=(D, h, freq=

In [119]:
def generatecode(node, current_code="", codes=None):
    if codes is None:
        codes = {}
    if isinstance(node.left, HuffmanNode) == False:
        codes[node.left] = current_code + "0"
    else:
        generatecode(node.left, current_code + "0", codes)
    
    if isinstance(node.right, HuffmanNode):
            generatecode(node.right, current_code+"1", codes)
    else:
        codes[node.right] = current_code + "1"
                                            
    return codes

    
    

In [120]:
encodingset1 = generatecode(huffmantree1)
print(encodingset1)

{'B': '0', 'A': '10', 'C': '110', 'E': '1110', 'D': '1111'}


In [121]:
encodingset2 = generatecode(huffmantree2)
print(encodingset2)

{',': '000000', 'q': '000001', 'p': '00001', 'l': '0001', 'i': '001', 'r': '0100', 'D': '01010000', 'h': '01010001', 'g': '0101001', 'b': '0101010', '.': '0101011', 'v': '0101100', 'x': '0101101', 'f': '0101110', 'E': '01011110', 'L': '010111110', 'U': '010111111', 'n': '0110', 'u': '0111', 'o': '1000', 'a': '1001', 't': '1010', 'c': '10110', 'm': '10111', ' ': '110', 's': '11100', 'd': '11101', 'e': '1111'}


In [122]:
def huffman_encode(message, codes):
    translationtable = str.maketrans(codes)
    encoded_message = message.translate(translationtable)
    return encoded_message
    

In [123]:
encodedset1 = huffman_encode(set1, encodingset1)
print(encodedset1)

1010101010100000000110110110111111111110


In [124]:
def huffman_decode(encoded_message, huffmantree):
    decodedstring = ""
    encoded_list = list(encoded_message)
    current_node = huffmantree
    for i in range(0, len(encoded_list)):
        if encoded_list[i] == "1":
            if isinstance(current_node.right, HuffmanNode):
                current_node = current_node.right
            else:
                decodedstring = decodedstring + current_node.right
                current_node = huffmantree
        else:
            if isinstance(current_node.left, HuffmanNode):
                current_node = current_node.left
            else:
                decodedstring = decodedstring + current_node.left
                current_node = huffmantree

    return decodedstring



In [125]:
huffman_decode(encodedset1, huffmantree1)

'AAAAAABBBBBBBCCCDDE'

In [126]:
encodedset2 = huffman_encode(set2, encodingset2)
print(encodedset2)

0101111101000010011111011111000100001111000111101111101110110000001100001001101110000110101101001101111111101000000011010110100001101110011111011010101111101001110100110100111101001000010011110010110001011001010011101111000100110100000001101110011111110111011101100011011110010111111001011110001110111010101111101110000110000100110001011010110001111010011110101110110101011001111010110000110010101010100001001111110111110101101110110000001100001001111110101111001010100101101001110100100010010000010111100101010111100101111111010110111101100011011111010011110111010111001011000110111110010110011110110001100110111000000110000001011100111100110011010001110010100100011111101110111101011011111010010110001101010011010001100001101100111000100011001101111011010001100001100101010101000010000111100110011000111100001110011110101101001000100100000101110010000111011110101101110111110011101011010001011110111100011101100011010110100001101110011110000010111100110100101011110010100000111001111001101001011110

In [127]:
huffman_decode(encodedset2, huffmantree2)

'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum'

Now to measure:
- Original size
- compressed size
- comparison to lower bound of entropy

In [128]:
originalbits_1 = len(set1) * 8 
originalbits_2 = len(set2) * 8 

compressed_bits_1 = len(encodedset1)
compressed_bits_2 = len(encodedset2)

#ratios
print(f"Set 1: {originalbits_1} bits -> {compressed_bits_1} bits ({compressed_bits_1/originalbits_1:.2f})")
print(f"Set 2: {originalbits_2} bits -> {compressed_bits_2} bits ({compressed_bits_2/originalbits_2:.2f})")


Set 1: 152 bits -> 40 bits (0.26)
Set 2: 3552 bits -> 1840 bits (0.52)


In [141]:


frequencies = convert_string_to_freq(set1)
probabilities = np.array(list(frequencies.values())) / sum(frequencies.values())
entropy_1 = entropy(probabilities)


print(entropy_1)

2.041814515501815


In [142]:
theoreticalminimumbits = entropy_1 * len(set1)
print(theoreticalminimumbits)

38.794475794534485


In [143]:
frequenciesset2 = convert_string_to_freq(set2)
probabilitiesset2 = np.array(list(frequenciesset2.values())) / sum(frequenciesset2.values())
entropy_2 = entropy(probabilitiesset2)
print(entropy_2)

4.110349293880954


In [144]:
theoreticalminimumbitsset2 = entropy_2 * len(set2)
print(theoreticalminimumbitsset2)

1824.9950864831435


In [145]:
print(f"Set 1: Huffman={len(encodedset1)}, Minimum={theoreticalminimumbits:.1f}, Overhead={len(encodedset1) - theoreticalminimumbits:.1f} bits")
print(f"Set 2: Huffman={len(encodedset2)}, Minimum={theoreticalminimumbitsset2:.1f}, Overhead={len(encodedset2) - theoreticalminimumbitsset2:.1f} bits")

Set 1: Huffman=40, Minimum=38.8, Overhead=1.2 bits
Set 2: Huffman=1840, Minimum=1825.0, Overhead=15.0 bits
